# ETL Pterodactyl Minecraft Application

### Index

- Install requierements
- Import libraries and setup key variables
- Setup directories, functions and folder creation
- Get Pterodactyl Application information
- Upload csv table files into Postgres

## Install requierements

In [ ]:
pip install -r requirements.txt

## Import libraries and setup key variables
Remember to add you own credentials in the .env file for them to be loaded here

In [1]:
import datetime, csv, os
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
from tuya_connector import TuyaOpenAPI
import pandas as pd

# Load .env file credentials
load_dotenv()

# Database connection
host = os.getenv('POSTGRES_HOST')
port = os.getenv('POSTGRES_PORT')
database = os.getenv('POSTGRES_DATABASE')
username = os.getenv('POSTGRES_USERNAME')
password = os.getenv('POSTGRES_PASSWORD')
connection = f'postgresql://{username}:{password}@{host}:{port}/{database}'

# Tuya connection
api_endpoint = os.getenv('API_ENDPOINT')
access_id = os.getenv('ACCESS_ID')
access_key = os.getenv('ACCESS_KEY')


# Connecto to Tuya API
openapi = TuyaOpenAPI(api_endpoint, access_id, access_key)
openapi.connect()

{'result': {'access_token': '8790265393fdcf776c0432c9205ba2ca',
  'expire_time': 5247,
  'refresh_token': 'bd1b217d15a6b24a99c55de20d1f3cb6',
  'uid': 'bay1692115764835sO5E'},
 'success': True,
 't': 1703788119116,
 'tid': 'eb9fd438a5ae11eea429ba1d23b51364'}

## Setup directories, functions and folder creation

## Get Tuya's devices information

In [3]:
uid = ''
project_id = ''
device_id = ''
#response = openapi.get(f'/v1.0/users/{uid}/devices') 
#response = openapi.get(f'/v1.0/osaas/projects/{project_id}')
response = openapi.get(f'/v1.0/devices/{device_id}')
print(response)

{'result': {'active_time': 1699992947, 'biz_type': 0, 'category': 'cz', 'create_time': 1699992947, 'icon': 'smart/icon/ay15422864509092y6k8/b14b6f478edbfbb218d5476f345306f4.png', 'id': 'eb0ea8501691a1a3315i1v', 'ip': '186.78.2.98', 'lat': '-33.0500', 'local_key': 'PAuUq/jU99D;VP[_', 'lon': '-71.6200', 'model': 'LSP-DL', 'name': 'DataCenter Karina', 'online': True, 'owner_id': '165624421', 'product_id': 'wrz6vzch8htux2zp', 'product_name': 'WiFi Plug', 'status': [{'code': 'switch_1', 'value': True}, {'code': 'countdown_1', 'value': 0}, {'code': 'add_ele', 'value': 41}, {'code': 'cur_current', 'value': 470}, {'code': 'cur_power', 'value': 984}, {'code': 'cur_voltage', 'value': 2288}, {'code': 'relay_status', 'value': 'last'}, {'code': 'overcharge_switch', 'value': False}, {'code': 'light_mode', 'value': 'relay'}, {'code': 'child_lock', 'value': False}, {'code': 'cycle_time', 'value': ''}, {'code': 'random_time', 'value': ''}, {'code': 'switch_inching', 'value': ''}], 'sub': False, 'time_z

## Upload csv table files into Postgres

In [ ]:
engine = create_engine(connection)

for file_table in os.listdir(server_app_folder):

    # Setup database variables
    ID = 'id'
    TABLE = file_table.split('.')[0]
    TABLE_UPDATE = TABLE + '_update'
    IS_ACTIVE_TABLE = 'is_active_table'
    SCHEMA = 'pterodactyl'
    SCHEMA_UPDATE = 'pterodactyl_update'
    
    # Reading of the file_table
    df = pd.read_csv(os.path.join(server_app_folder, file_table))

    # Start connection with database
    with engine.connect() as conn:
        # Start a new transaction
        trans = conn.begin()

        try:
            # Load ID from database
            result = conn.execute(text(f'SELECT "{ID}" FROM {SCHEMA}.{TABLE}'))
            db = pd.DataFrame(result.fetchall(), columns=result.keys())

            # Compare ID
            sameID = db[ID].isin(df[ID])

            toUpdate = df[df[ID].isin(db[ID][sameID])]
            toIngest = df[~df[ID].isin(db[ID][sameID])]
            toDelete = db[~db[ID].isin(df[ID])]

            # Insert the DataFrame into a table
            toIngest.to_sql(TABLE, conn, schema=SCHEMA, if_exists='append', index=False)

            # Insert the updatable DataFrame into the TABLE_UPDATE table
            toUpdate.to_sql(TABLE_UPDATE, conn, schema=SCHEMA_UPDATE, if_exists='append', index=False)

            # Define and execute the following queries
            conn.execute(text(f'DELETE FROM {SCHEMA}.{TABLE} WHERE "{ID}" IN (SELECT "{ID}" FROM {SCHEMA_UPDATE}.{TABLE_UPDATE});'))
            conn.execute(text(f'INSERT INTO {SCHEMA}.{TABLE} SELECT * FROM {SCHEMA_UPDATE}.{TABLE_UPDATE};'))
            conn.execute(text(f'TRUNCATE TABLE {SCHEMA_UPDATE}.{TABLE_UPDATE};'))

            # Update column "is_active" from tables when data is deleted from Pterodactyl App
            toDelete.to_sql(IS_ACTIVE_TABLE, conn, schema=SCHEMA_UPDATE, if_exists='append', index=False)
            conn.execute(text(f'UPDATE {SCHEMA}.{TABLE} SET is_active = false WHERE "{ID}" IN (SELECT * FROM {SCHEMA_UPDATE}.{IS_ACTIVE_TABLE});'))
            conn.execute(text(f'TRUNCATE TABLE {SCHEMA_UPDATE}.{IS_ACTIVE_TABLE};'))

            # Commit the transaction
            trans.commit()

        except Exception as e:
            # Rollback the transaction on exception
            print('!!! [ERROR IN DATABASE QUERIES] !!!')
            trans.rollback()
            print('Transaction has been rolled back')
            print(f'Error occurred during transaction:\n{e}')
            raise

with engine.connect() as conn:
    # Start a new transaction
    trans = conn.begin()

    try:
        # Update date from the last_update table based on max date on the file
        old_last_update = conn.execute(text(f'SELECT date FROM {SCHEMA}.last_update')).fetchall()[0][0]
        new_last_update = last_update
        if new_last_update > old_last_update:
            conn.execute(text(f"UPDATE {SCHEMA}.last_update SET date = '{new_last_update}';"))

        # Commit the transaction
        trans.commit()

    except Exception as e:
        # Rollback the transaction on exception
        print('!!! [ERROR IN DATABASE QUERIES] !!!')
        trans.rollback()
        print('Transaction has been rolled back')
        print(f'Error occurred during transaction:\n{e}')
        raise